In [1]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go
from multiprocess import Pool

In [2]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
run3_system = dataframes_from_spreadsheet("cms_system_60.xlsx")
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")
run5_smartpx_system = dataframes_from_spreadsheet("cms_system_200_smartpx.xlsx")

In [3]:
run3_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Link Efficiency (J/bit),Op Efficiency (J/op),Compression
0,Tracking,Inner Tracker,436666.666667,40000000,2.220000e-11,0,0
1,Tracking,Outer Tracker PS,206666.666667,40000000,2.220000e-11,0,0
2,Tracking,Outer Tracker 2S,126666.666667,40000000,2.220000e-11,0,0
3,Tracking,Track Finder TPG,10000.000000,40000000,2.220000e-11,0,0
4,Timing,MIP Timing BTL,76666.666667,40000000,2.220000e-11,0,0
5,Timing,MIP Timing ETL,136666.666667,40000000,2.220000e-11,0,0
6,Calorimetry,ECAL Barrel,180000.000000,40000000,2.220000e-11,0,0
7,Calorimetry,HCAL Barrel,240000.000000,40000000,2.220000e-11,0,0
8,Calorimetry,HCAL HO,30000.000000,40000000,2.220000e-11,0,0
9,Calorimetry,HCAL HF,60000.000000,40000000,2.220000e-11,0,0


In [4]:
run3_system.triggers

,Name,Output,Data (bytes),Reduction Ratio,Classifier,Skill mean,Skill variance,Link Efficiency (J/bit),Op Efficiency (J/op),Compression
0,Tracking,Intermediate,0,1,Dummy,0,0,2.500000e-11,0.000,0
1,Timing,Intermediate,0,1,Dummy,0,0,2.500000e-11,0.000,0
2,Calorimetry,Intermediate,0,1,Dummy,0,0,2.500000e-11,0.000,0
3,Muon,Intermediate,0,1,Dummy,0,0,2.500000e-11,0.000,0
4,Intermediate,Global,260000,400,L1T,0,0,2.500000e-11,0.003,0
5,Global,Disk,0,100,HLT,4,1,2.500000e-11,16.000,0
6,Disk,NaN,0,1,Dummy,0,0,2.500000e-11,0.000,0


In [5]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [6]:
[d for d in run5_system.detectors.iloc][9]

Category                   Calorimetry
Detector                       HCAL HF
Data (bytes)                     60000
Sample Rate                   40000000
Compression                          0
Link Efficiency (J/bit)            0.0
Op Efficiency (J/op)                 0
PU 200                            0.06
Name: 9, dtype: object

In [7]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])

In [8]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}

In [9]:
"""
Vary the accept rate of the level 1 trigger and inspect its impact on performance and resources required
"""
def init_system(functions, l1t_reduction: float, pileup_interp: float):
    d_3 = run3_system.detectors.copy()

    new_vals = (1 - pileup_interp) * d_3["Data (bytes)"].values + (pileup_interp) * run5_system.detectors["Data (bytes)"].values
    d_3["Data (bytes)"] = new_vals
        
    t = run3_system.triggers.copy()
    #intermediate reduction stage
    t.at[4, "Reduction Ratio"] = l1t_reduction

    g = construct_graph(d_3, t, run3_system.globals, functions)

    return g

In [10]:
ex_baseline = init_system(funcs, 400, 0.0)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])
/Users/wilkie/code/system_flow/classifier.py:424: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probabl

In [11]:
ex_baseline.graph["op power"] / 1e6

2.0689478234103813

In [12]:
ex_pu200 = init_system(funcs, 400, 1.0)

/Users/wilkie/code/system_flow/classifier.py:200: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])
/Users/wilkie/code/system_flow/classifier.py:424: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probabl

In [13]:
ex_pu200.graph["op power"] / 1e6

45.229706543481164

In [14]:
from copy import deepcopy

In [15]:
def extract_results(graph):

    power = graph.graph["op power"] + graph.graph["link power"]
    confusion = graph.graph["performance"]

    return power, confusion

In [16]:
extract_results(ex_baseline)

(2095279.3794481144,
 array([[39998227,      768],
        [     771,      230]]))

In [17]:
extract_results(ex_pu200)

(45353201.34174617,
 array([[39998217,      779],
        [     781,      219]]))

In [18]:
def vary_pileup(graph, interp: float):
    for (i, d) in enumerate(run3_system.detectors.iloc):
        name = d["Detector"]
        #interpolate linearly between run3 and run5 data rates
        data = (1 - interp) * d["Data (bytes)"] + (interp) * run5_system.detectors.iloc[i]["Data (bytes)"]
        graph.nodes[name]["sample data"] = data

    return graph

In [19]:
def vary_system(graph, reduction_ratio: float, interp):
    graph = deepcopy(graph)
    graph.nodes["Intermediate"]["reduction ratio"] = reduction_ratio
    graph = vary_pileup(graph, interp)
    

    graph = update_throughput(graph)

    power = graph.graph["op power"] + graph.graph["link power"]
    confusion = graph.graph["performance"]

    return power, confusion

In [20]:
baseline = vary_system(ex_baseline, 400, 0.0)

In [21]:
baseline

(2095279.3794481144,
 array([[39998227,      768],
        [     771,      230]]))

In [22]:
run5 = vary_system(ex_baseline, 53.3, 1.0)

In [23]:
run5

(335489601.7178109,
 array([[39987966,     4525],
        [    4527,     2977]]))

In [24]:
#vary this accept rate from today's rate to the planned Run-5 
l1t_reductions = np.linspace(450, 40, 101)
pileup = np.linspace(0.01, 1.0, 101)

In [25]:
pmap_args = []
for s in pileup:
    for r in l1t_reductions:
        pmap_args.append((ex_baseline, r, s))

In [26]:
def map_fn(x):
    return vary_system(x[0], x[1], x[2])

In [27]:
with Pool(6) as p:
    res = p.map(map_fn, pmap_args)

In [28]:
res2 = [res[i:i+len(pileup)] for i in range(0, len(pileup)*len(l1t_reductions), len(pileup))]

In [29]:
def sys_productivity(confusion, power):
    n = np.sum(get_passed(confusion))
    f1 = f1_score(confusion)
    productivity = (n * f1) / power
    return productivity

In [30]:
def extract_metrics(results):
    all_confusion = np.array([r[1] for r in results])

    all_power = [r[0] / density_scale_model(2032) for r in results]
    all_power = np.array(all_power)

    all_recall = np.array([recall(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    all_precision = np.array([precision(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    all_f1 = np.array([f1_score(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])

    all_productivity = [sys_productivity(all_confusion[i,:,:], all_power[i]) for i in range(all_confusion.shape[0])]

    metrics = {"confusion": all_confusion,
               "power": all_power,
               "recall": all_recall,
               "precision": all_precision,
               "f1 score": all_f1,
               "productivity" : all_productivity}

    return metrics

In [31]:
run5_metrics = [extract_metrics(r) for r in res2]

In [32]:
res_f1 = np.stack([r["f1 score"] for r in run5_metrics]).transpose()

In [33]:
res_recall = np.stack([r["recall"] for r in run5_metrics]).transpose()

In [34]:
res_precision = np.stack([r["precision"] for r in run5_metrics]).transpose()

In [35]:
power = np.stack([r["power"] for r in run5_metrics])

In [36]:
power[1,1]

326691.65589805925

In [37]:
res_productivity = np.stack([r["productivity"] for r in run5_metrics])

In [38]:
from scipy.ndimage import gaussian_filter

In [56]:
smoothed_f1 = gaussian_filter(res_f1, sigma=4)

In [57]:
#np.savez_compressed("smoothed_f1.npz", smoothed_f1)

In [58]:
fig = go.Figure(data =
    go.Contour(
        z=smoothed_f1,
        x=l1t_reductions, # horizontal axis
        y=pileup, # vertical axis,
        contours = dict(showlabels = True),
        colorbar = dict(title = "F1 Score")
         
    ),
    )

y_offset = 0.015
fig.add_trace(go.Scatter(x = (400,),
                        y = (0.0 + y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "gray", symbol="circle"),
                        name = "Phase-1"))

fig.add_trace(go.Scatter(x = (53.3,),
                        y = (1.0 - y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "gray", symbol = "star"),
                        name = "Phase-2"))

fig.update_layout(width = 800, 
                  height = 600,
                  xaxis_title = "L1T Reduction Ratio",
                  yaxis_title = "Pileup",
                  title = "F1 Score by Pileup & Reduction Ratio",
                  legend=dict(xanchor = "right",
                    x = 0.95))
fig.update_xaxes(autorange="reversed")
fig.update_yaxes(range=[0.0, 0.8])
fig.show()

In [59]:
fig = go.Figure(data =
    go.Contour(
        z=power,
        x=l1t_reductions, # horizontal axis
        y=pileup, # vertical axis,
        contours = dict(showlabels = True),
        colorbar = dict(title = "Power (W)")
         
    ),
    )

y_offset = 0.015
fig.add_trace(go.Scatter(x = (400,),
                        y = (0.0 + y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "gray", symbol="circle"),
                        name = "Phase-1"))

fig.add_trace(go.Scatter(x = (53.3,),
                        y = (1.0 - y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "gray", symbol = "star"),
                        name = "Phase-2"))

fig.update_layout(width = 800, 
                  height = 600,
                  xaxis_title = "L1T Reduction Ratio",
                  yaxis_title = "Pileup",
                  title = "DAQ Power by Pileup & Reduction Ratio",
                  legend=dict(xanchor = "right",
                    x = 0.20,
                    y = 0.95))
fig.update_xaxes(autorange="reversed")
fig.update_yaxes(range=[0.0, 0.8])
fig.show()

In [60]:
fig = go.Figure(data =
    go.Contour(
        z=res_productivity * 1000,
        x=l1t_reductions, # horizontal axis
        y=(pileup* 140)+60, # vertical axis,
        contours = dict(showlabels = True),
        colorbar = dict(title = "Productivity (1/kJ)")
         
    ),
    )

y_offset = 3
fig.add_trace(go.Scatter(x = (400,),
                        y = (60 + y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "green", symbol="circle"),
                        name = "Run-3"))

fig.add_trace(go.Scatter(x = (53.3,),
                        y = (200 - y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "green", symbol = "star"),
                        name = "Run-5"))

fig.update_layout(width = 800, 
                  height = 600,
                  xaxis_title = "L1T Reduction Ratio",
                  yaxis_title = "Pileup",
                  title = "System Productivity by Pileup & L1T Reduction Ratio",
                  legend=dict(xanchor = "right",
                   x = 0.20,
                y = 0.95),)
fig.update_xaxes(autorange="reversed")
fig.add_annotation(x = -0.1, 
                   y = -0.1, 
                   showarrow=False,
                   text = "Baseline System (2032)", 
                   xref="paper", 
                   yref="paper",
                   font = dict(size = 14))
fig.show()

In [44]:
fig = go.Figure(data =
    go.Contour(
        z=smoothed_f1,
        x=l1t_reductions, # horizontal axis
        y=pileup, # vertical axis,
        contours = dict(showlabels = True),
        colorbar = dict(title = "F1 Score")
         
    ),
    )

y_offset = 0.015
fig.add_trace(go.Scatter(x = (400,),
                        y = (0.0 + y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "gray", symbol="circle"),
                        name = "Phase-1"))

fig.add_trace(go.Scatter(x = (53.3,),
                        y = (1.0 - y_offset,),
                        mode = "markers",
                        marker = dict(size = 14, color = "gray", symbol = "star"),
                        name = "Phase-2"))

fig.update_layout(width = 800, 
                  height = 600,
                  xaxis_title = "L1T Reduction Ratio",
                  yaxis_title = "Pileup",
                  title = "F1 Score by Pileup & Reduction Ratio",
                  legend=dict(xanchor = "right",
                    x = 0.95))
fig.update_xaxes(autorange="reversed")
fig.update_yaxes(range=[0.0, 0.8])
fig.show()

In [45]:
#because its rejection is so much higher, there's more potential improvement gained by making L1T's skill higher 
#than simply passing more data to the HLT